In [ ]:
!pip install deap
!pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import numpy as np
w = np.asarray([[0,1],[2,1]])
w[1,0]

2

**Dataset:** https://archive.ics.uci.edu/ml/datasets/Car+Evaluation

**Algorithm:** XGBoost 

In [ ]:
import numpy as np
from pandas import read_csv

url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data'

data = read_csv(url, header=None, usecols=range(0, 7))
col_names = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety','class']
data.columns = col_names
X = data.iloc[:, 0:6]
y = data.iloc[:, 6:7]

X

,buying,maint,doors,persons,lug_boot,safety
0,vhigh,vhigh,2,2,small,low
1,vhigh,vhigh,2,2,small,med
2,vhigh,vhigh,2,2,small,high
3,vhigh,vhigh,2,2,med,low
4,vhigh,vhigh,2,2,med,med
...,...,...,...,...,...,...
1723,low,low,5more,more,med,med
1724,low,low,5more,more,med,high
1725,low,low,5more,more,big,low
1726,low,low,5more,more,big,med


In [ ]:
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder

encoder = ce.OrdinalEncoder(cols=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety'])
X = encoder.fit_transform(X)

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
y

array([2, 2, 2, ..., 2, 1, 3])

In [ ]:
# Define the XGBoost parameters
params = {
    'objective': ['multi:softmax','multi:softprob','multi:logloss','binary:hinge'],
    'n_estimators': [10,100,500],
    'eta': np.logspace(-2, 0, num=10, base=10),
    'max_depth': [3,6,9,12]
}

In [ ]:
from deap import base
from deap import creator
from deap import tools
import random

# define a single objective, maximizing fitness strategy:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
# create the Individual class based on list:
creator.create("Individual", list, fitness=creator.FitnessMax)
toolbox = base.Toolbox()
# fill in the with random floats for each type of hyper parameter
toolbox.register("h1", random.randint,0,3)
toolbox.register("h2", random.randint,0,2)
toolbox.register("h3", random.uniform,0.01,1.0)
toolbox.register("h4", random.randint,0,3)

toolbox.register("IndividualCreator",tools.initCycle,creator.Individual,
                 (toolbox.h1,toolbox.h2,toolbox.h3,toolbox.h4),n=1)

/usr/local/lib/python3.10/dist-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [ ]:
def getParams(individual):
    objective = ['multi:softmax','multi:softprob','multi:logloss','binary:hinge'][round(individual[0])]
    n_estimators =  [10,100,500][round(individual[1])] 
    eta = individual[2]
    max_depth = [3,6,9,12][round(individual[3])]
    return objective, n_estimators, eta,max_depth

In [ ]:
# Test IndividualCreator
for i in range(10):
    a = toolbox.IndividualCreator()
    b = getParams(a)
    print(a, b)

[1, 1, 0.6311957143746805, 3] ('multi:softprob', 100, 0.6311957143746805, 12)
[3, 2, 0.1491426839735897, 1] ('binary:hinge', 500, 0.1491426839735897, 6)
[1, 1, 0.38841991255327324, 0] ('multi:softprob', 100, 0.38841991255327324, 3)
[0, 1, 0.18895369200110504, 0] ('multi:softmax', 100, 0.18895369200110504, 3)
[2, 2, 0.9739385522185579, 1] ('multi:logloss', 500, 0.9739385522185579, 6)
[0, 2, 0.5759915475977623, 2] ('multi:softmax', 500, 0.5759915475977623, 9)
[0, 2, 0.4147699863179217, 1] ('multi:softmax', 500, 0.4147699863179217, 6)
[0, 2, 0.10890832470969909, 1] ('multi:softmax', 500, 0.10890832470969909, 6)
[1, 0, 0.7523226713663165, 2] ('multi:softprob', 10, 0.7523226713663165, 9)
[0, 0, 0.1413729760408519, 1] ('multi:softmax', 10, 0.1413729760408519, 6)


In [ ]:
from sklearn import model_selection
from xgboost import XGBClassifier

kfold = model_selection.KFold(n_splits=10, random_state=42, shuffle=True)

def getAccuracy(individual):
    objective, n_estimators, eta,max_depth = getParams(individual)
    classifier = XGBClassifier(random_state=42,
                                         objective=objective,
                                         n_estimators=n_estimators,
                                         eta=eta,
                                        max_depth=max_depth
                                        #num_class=2
                                         )

    cv_results = model_selection.cross_val_score(classifier,
                                                 X,
                                                 y,
                                                 cv=kfold,
                                                 scoring='accuracy')
    return cv_results.mean()

In [ ]:
# Get the accuracy from default hyper-parameters setup

# objective = 'binary:logistic'
# eval_metric = None
# eta = 0.1
# max_depth = 3
kfold = model_selection.KFold(n_splits=10, random_state=42, shuffle=True)
classifier = XGBClassifier()
cv_results = model_selection.cross_val_score(classifier,
                                                 X,
                                                 y,
                                                 cv=kfold,
                                                 scoring='accuracy')

print('mean: ',cv_results.mean())

mean:  0.9918974324505981


In [ ]:
# create the population operator to generate a list of individuals:
toolbox.register("populationCreator", tools.initRepeat, list, 
                 toolbox.IndividualCreator)
# fitness calculation
def classificationAccuracy(individual):
    return getAccuracy(individual),

toolbox.register("evaluate", classificationAccuracy)

# genetic operators:
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutPolynomialBounded, eta=1.0, low=[0, 0, 0.01, 0], up=[3, 2, 1.0, 3], indpb=0.3)
toolbox.register("select", tools.selTournament, tournsize=3)

In [ ]:
individual = [2, 0, 0.41245413088983973, 3]
toolbox.mutate(individual)

([1.9161021009252097, 0, 0.41245413088983973, 3],)

In [ ]:
from deap import algorithms

# Genetic Algorithm constants:
POPULATION_SIZE = 20
P_CROSSOVER = 0.9  # probability for crossover
P_MUTATION = 0.5   # probability for mutating an individual
MAX_GENERATIONS = 5
HALL_OF_FAME_SIZE = 5

# create initial population (generation 0):
population = toolbox.populationCreator(n=POPULATION_SIZE)

# prepare the statistics object:
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("max", np.max)
stats.register("avg", np.mean)

# define the hall-of-fame object:
hof = tools.HallOfFame(HALL_OF_FAME_SIZE)

# perform the Genetic Algorithm flow with hof feature added:
population, logbook = algorithms.eaSimple(population,
                                          toolbox,
                                          cxpb=P_CROSSOVER,
                                          mutpb=P_MUTATION,
                                          ngen=MAX_GENERATIONS,
                                          stats=stats,
                                          halloffame=hof,
                                          verbose=True)

# print best solution found:
print("- Best solution is: ")
print("params = ", hof.items[0], getParams(hof.items[0]))
print("Accuracy = %1.5f" % hof.items[0].fitness.values[0])

gen	nevals	max     	avg     
0  	20    	0.994798	0.984783
1  	20    	0.994798	0.992568
2  	20    	0.994798	0.993435
3  	20    	0.994798	0.993899
4  	19    	0.995376	0.994217
5  	20    	0.995376	0.994536
- Best solution is: 
params =  [1.8721773541497626, 1.7325461983941781, 0.8562246916483003, 1] ('multi:logloss', 500, 0.8562246916483003, 6)
Accuracy = 0.99538
